# Capstone Project
This project will be to make a platform that will use machine learning to classify hand shapes that are used in ASL. 

Here are a few of the prerequisites required on your machine or virtual environment to get the projecy working:

* Python 3.8+
* Use PIP to install the following packages:
    * sklearn
    * PIL
    * numpy
    * imutils

## Importing the packages. 

The first thing we need to do, after installing, is to import the packages

In [9]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
from imutils import paths
import numpy as np
import os
import pickle

## Notes

Because we are working with images, we need to split the image into its respective RGB color channels, and then create a feature vector with 6 values: *The mean and standard deviation for each of the 3 channels*.

In [10]:
# Function for changing the colorgrade
def extract_color_statistics(image):
    (R, G, B) = image.split()
    features = [np.mean(R), np.mean(G), np.mean(B), np.std(R), np.std(G), np.std(B)]

    return features

## Variables and data/labels

The cell below is where we will initialize our variables.

*dataset* - This is the path to our image folder
*imagePath* - Use the **imutils** import to set the path for our code
*data* - The data list will hold all our image *feature vectors*.
*labels* - The labels list will hold the corresponding labels

In [11]:
# Set and initialize variables for later use
dataset = "Dataset"
imagePaths = paths.list_images(dataset)
data = []
labels = []

## Our database

Now that we have our path set, it is time to run the images through our previous **extract_color_statistics** function.

The for loop will behave as such:

* Load the input image
* Compute and extract the channel stats
* Add the data to the **data** list
* Extract the label class from the path name
* Update the labels list

In [12]:
# loop over our input images
for imagePath in imagePaths:
    # Load the input image from disk
    image = Image.open(imagePath)
    # Extract the features
    features = extract_color_stats(image)
    data.append(features)
    # extract the class label from the file path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

## Our labels

Now that the loop is completed, we need to encode our labels, converting them from strings to integers.

In [13]:
le = LabelEncoder()
labels = le.fit_transform(labels)